# 2. The Importance of Linearity in Finance

$$N(x) = \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{x} e^{- \frac{z^2}{2}}\, dz$$


### CAPM Expected Return with SML

SML function :$$E(R)=R +β E(R )−R $$

In [1]:
from scipy import stats
stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err =  stats.linregress(stock_returns, mkt_returns)
print (beta, alpha)

0.5077431878770808 -0.008481900352462384


###  OLS Regression Model

In [4]:
import numpy as np
import statsmodels.api as sm
# Generate some sample data
num_periods = 9
all_values  = np.array([np.random.random(8)
                           for i in range(num_periods)])
# Filter the data
y_values = all_values[:, 0] # First column values as Y 
x_values = all_values[:, 1:] # All other values as X
x_values = sm.add_constant(x_values) # Include the intercept 
results  = sm.OLS(y_values, x_values).fit() # Regress andfit the model
print (results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     88.66
Date:                Fri, 17 Apr 2020   Prob (F-statistic):             0.0816
Time:                        12:51:44   Log-Likelihood:                 31.272
No. Observations:                   9   AIC:                            -46.54
Df Residuals:                       1   BIC:                            -44.97
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2552      0.084     14.893      0.0

/Applications/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))


In [5]:
print (results.params)

[ 1.25521363 -0.67256257 -1.06754757  0.96674982 -0.53727054 -0.12350831
  0.16264335 -0.35107298]


### Linear Optimization

In [8]:
import pulp
x = pulp.LpVariable("x", lowBound=0)  # x ≥ 0, y ≥ 0
y = pulp.LpVariable("y", lowBound=0)  # x ≥ 0, y ≥ 0
problem = pulp.LpProblem("A simple maximization objective", pulp.LpMaximize)
problem += 3*x + 2*y, "The objective function" # Maximize: f(x,y) = 3x+2y
problem += 2*x + y <= 100, "1st constraint" # 2x + y ≤ 100
problem += x + y <= 80, "2nd constraint"  # x + y ≤ 80
problem += x <= 40, "3rd constraint"  # x ≤ 40
problem.solve()

print ("Maximization Results:")
for variable in problem.variables():
    print (variable.name, "=", variable.varValue)

Maximization Results:
x = 20.0
y = 60.0


### Integer Programming

Suppose we must go for 150 contracts in a particular over-the-counter exotic security from three dealers. Dealer X quoted $500 per contract plus handling
fees of $4,000, regardless of the number of contracts sold. Dealer Y charges $450 per contract plus a transaction fee of $2,000. Dealer Z charges $450 per contract plus a fee of $6,000. Dealer X will sell at most 100 contracts, dealer Y at most 90, and dealer Z at most 70. The minimum transaction volume from any dealer is 30 contracts if any are transacted with that dealer. How should we minimize the cost of purchasing 150 contracts?

In [10]:
import pulp
dealers = ["X", "Y", "Z"] 
variable_costs = {"X": 500, "Y": 350,"Z": 450} 
fixed_costs = {"X": 4000, "Y": 2000,"Z": 6000}
# Define PuLP variables to solve
quantities = pulp.LpVariable.dicts("quantity",dealers,
                                   lowBound=0, cat=pulp.LpInteger)
is_orders = pulp.LpVariable.dicts("orders", dealers,
                                  cat=pulp.LpBinary)

$$Minimize∑ IsOrder [variable cost × quantity + fixed cost ]$$

$$Minimize∑ IsOrder [variable cost × quantity + fixed cost ]$$

$$i=x$$

$$IsOrder = i$$

$$ 1, if buying from dealer i $$
$$0,if not buying from dealer i$$

$$30≤quantityx ≤100$$

$$30 ≤ quantityy ≤ 90$$

$$30≤quantityz ≤70$$


$$∑i=z quantity i =150$$
$$i=x$$

In [27]:
"""
   This is an example of implementing an integer programming model with
binary
variables
the wrong way.
"""
# Initialize the model with constraints
model = pulp.LpProblem("A cost minimization problem", pulp.LpMinimize)
model += sum([(variable_costs[i] * quantities[i] +fixed_costs[i])*
              is_orders[i] for i in dealers]) #"Minimize portfolio cost"
model += sum([quantities[i] for i in dealers]) == 150 # "Total contracts required"

model += 30 <= quantities["X"] <= 100, "Boundary of total volume of X"
model += 30 <= quantities["Y"] <= 90, "Boundary of total volume of Y"
model += 30 <= quantities["Z"] <= 70, "Boundary of total volume of Z"
model.solve()

'\n   This is an example of implementing an integer programming model with\nbinary\nvariables\nthe wrong way.\n\n# Initialize the model with constraints\nmodel = pulp.LpProblem("A cost minimization problem", pulp.LpMinimize)\nmodel += sum([(variable_costs[i] * quantities[i] +fixed_costs[i])*\n              is_orders[i] for i in dealers]) #"Minimize portfolio cost"\nmodel += sum([quantities[i] for i in dealers]) == 150 # "Total contracts required"\nmodel += 30 <= quantities["X"] <= 100, "Boundary of total volume of X"\nmodel += 30 <= quantities["Y"] <= 90, "Boundary of total volume of Y"\nmodel += 30 <= quantities["Z"] <= 70, "Boundary of total volume of Z"\nmodel.solve()\n'

### A different approach with binary conditions

$$Minimize∑variable cost × quantity + fixed cost × IsOrder$$
$$i=x$$

Comparing with the previous objective equation, we would obtain the same fixed cost values. However, the unknown variable quantityi remains in the first term of the equation. Hence, the variable quantityi is required to be solved as a function of
such that the constraints are stated as follows:

$$IsOrder×30≥quantity ≤IsOrder×100 ixi$$

$$IsOrder×30≤quantity ≤IsOrder×90 iyi$$

$$IsOrder×30≤quantity ≤IsOrder×70 izi$$


In [31]:
"""
This is an example of implementing an IP model with binary variables the correct way.
"""
# Initialize the model with constraints
model = pulp.LpProblem("A cost minimization problem", pulp.LpMinimize)
model += sum([variable_costs[i]*quantities[i] + fixed_costs[i]*
              is_orders[i] for i in dealers]) # "Minimize portfolio cost"
model +=sum([quantities[i] for i in dealers]) == 150 # "Total contracts required"
model +=is_orders["X"]*30 <= quantities["X"] <= is_orders["X"]*100 # "Boundary of total volume of X"
model +=is_orders["Y"]*30 <= quantities["Y"] <= is_orders["Y"]*90 # "Boundary of total volume of Y"
model +=is_orders["Z"]*30 <= quantities["Z"] <= is_orders["Z"]*70 # "Boundary of total volume of Z"
model.solve()
print ("Minimization Results:")
for variable in model.variables():
    print (variable, "=", variable.varValue)
print ("Total cost: %s" % pulp.value(model.objective))

Minimization Results:
orders_X = 0.0
orders_Y = 1.0
orders_Z = 1.0
quantity_X = 0.0
quantity_Y = 90.0
quantity_Z = 60.0
Total cost: 66500.0


### Solving linear equations using matrices

Suppose we would like to build a portfolio consisting of 3 securities, a, b and c. The allocation of the portfolio must meet certain constraints: it must consist of 6 units of a long position in security a. With every 2 units of security a, 1 unit of security b, and 1 unit of security c invested, the net position must be 4 units long. With every 1 unit of security a, 3 units of security b, and 2 units of security c invested, the net position must be long 5 units.

To find out the number of securities to invest in, we can frame the problem mathematically as follows:
$$2a + b + c = 4$$ 
$$a + 3b + 2c = 5$$ 
$$a=6$$

With all of the coefficients visible, the equations are as follows:
$$2 a + 1b + 1c = 4$$
$$1a + 3b + 2c = 5$$
$$1a + 0b + 0c = 6$$

Taking the coefficients of the equations and representing them in a matrix form:
$$A=\begin{bmatrix} 1 & 2 & 1 \\ 3 & 0 & 1 \\ 0 & 2 & 4 \end{bmatrix},x=\begin{bmatrix} a \\ b \\ c \end{bmatrix},B=\begin{bmatrix} 4 \\ 5 \\ 6 \end{bmatrix}$$

The linear equations can now be stated as follows:
$$Ax = B$$
To solve for the vector x that contains the number of securities to invest in, the
inverse of matrix A is taken and the equation is written as follows: $$x = A^-1B$$

In [36]:
""" Linear algebra with NumPy matrices """
import numpy as np
A = np.array([[2, 1, 1], [1, 3, 2], [1, 0, 0]]) 
B = np.array([4, 5, 6])
print (np.linalg.solve(A, B ))

[  6.  15. -23.]


### The LU decomposition

The LU decomposition, or also known as lower upper factorization, is one of the methods of solving square systems of linear equations.
Product of two matrices: a lower triangular matrix L and an upper triangular matrix U.

$$A = LU$$

$$\begin{bmatrix} a & b & c \\ d & e & f \\ g & h & i \end{bmatrix}=
\begin{bmatrix} l_{11} & 0 & 0 \\ l_{21} & l_{22} & 0 \\ l_{31} & l_{32} & l_{33} \end{bmatrix}*
\begin{bmatrix} u_{11} & u_{12} & u_{13} \\ 0 & u_{22} & u_{23} \\ 0 & 0 & u_{33} \end{bmatrix}
$$

In [38]:
""" LU decomposition with SciPy """ 
import scipy.linalg as linalg 
import numpy as np
A = np.array([[2., 1., 1.], [1., 3., 2.], [1., 0., 0.]])
B = np.array([4., 5., 6.])
LU = linalg.lu_factor(A)
x = linalg.lu_solve(LU, B)
print(x)

[  6.  15. -23.]


In [47]:
P, L, U = linalg.lu(A)
print(P)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


### The Cholesky decomposition

the matrix A is decomposed as A = LL , L
is a lower triangular matrix with real and positive numbers on the diagonals, and LT is the conjugate transpose of L.

The equation is in the form of Ax = B, where A and B take the following values:

$$
A=\begin{bmatrix} 10& -1& 2& 0 \\ -1& 11& -1& 3 \\ 2& -1& 10& -1 \\ 0& 3& -1& 8 \end{bmatrix},
x=\begin{bmatrix} a \\ b \\ c \\ d \end{bmatrix},
B=\begin{bmatrix} 6 \\ 25 \\ -11 \\ 15 \end{bmatrix}
$$

In [52]:
""" Cholesky decomposition with NumPy """
import numpy as np
 
A = np.array([[10., -1., 2., 0.],
              [-1., 11., -1., 3.], 
              [2., -1., 10., -1.], 
              [0.0, 3., -1., 8.]]) 

B = np.array([6., 25., -11., 15.])

L = np.linalg.cholesky(A)

print (L)

[[ 3.16227766  0.          0.          0.        ]
 [-0.31622777  3.3015148   0.          0.        ]
 [ 0.63245553 -0.24231301  3.08889696  0.        ]
 [ 0.          0.9086738  -0.25245792  2.6665665 ]]


In [55]:
"""
We can use the definition of the Cholesky factorization by multiplying
L with its conjugate transpose that will lead us back to the values of matrix A:
"""
print (np.dot(L, L.T.conj())) # A=L.L*

[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]


In [56]:
# Before solving for x, we need to solve for LT x as y.
y = np.linalg.solve(L, B) # L.L*.x=B; When L*.x=y, then L.y=B

# To solve for x, all we need to do is to solve again using 
#                                  the conjugate transpose of L and y:
x = np.linalg.solve(L.T.conj(), y) # x=L*'.y

print (x)

[ 1.  2. -1.  1.]


In [57]:
print (np.mat(A) * np.mat(x).T) # B=Ax

[[  6.]
 [ 25.]
 [-11.]
 [ 15.]]


### The QR decomposition

The QR algorithm is commonly used to solve the linear least squares problem.

An orthogonal matrix exhibits the following properties:
• It is a square matrix
• Multiplying an orthogonal matrix by its transpose returns the identity matrix:
$$QQT =QTQ=1$$

• The inverse of an orthogonal matrix equals its transpose:
$$QT =Q−1$$


**An identity matrix is also a square matrix with its main diagonal containing ones and zeros elsewhere.**

We can now restate the problem Ax = B as follows: 

$$QRx = B$$

$$Rx = Q−1B or Rx = QT B$$

In [63]:
""" QR decomposition with scipy """ 
import scipy.linalg as linalg 
import numpy as np
A = np.array([ [2., 1., 1.], [1., 3., 2.], [1., 0., 0]])
B = np.array([4., 5., 6.])
Q, R = linalg.qr(A) # QR decomposition 
y = np.dot(Q.T, B) # Let y=Q'.B
x = linalg.solve(R, y) # Solve Rx=y
print(x)

[  6.  15. -23.]


### The Jacobi method

$$
A=\begin{bmatrix} a& b& c& d \\ e& f& g& h \\ i& j& k& l \\ m& n& o& p \end{bmatrix}=
\begin{bmatrix} a& 0& 0& 0 \\ 0& f& 0& 0 \\ 0& 0& k& 0 \\ 0& 0& 0& p \end{bmatrix}*
\begin{bmatrix} 0& b& c& d \\ e& 0& g& h \\ i& j& 0& l \\ m& n& o& 0 \end{bmatrix}
$$

The solution is then obtained iteratively as follows:
    
$$Ax = B$$

$$(D + R)x = B$$

$$Dx = B − Rx$$

$$x_{n+1} = D^1(B−Rx_n)$$

In [73]:
""" Solve Ax=B with the Jacobi method """ 
import numpy as np

def jacobi(A, B, n, tol=1e-10):
# Initializes x with zeroes with same shape and type as B 
    x = np.zeros_like(B)
    for it_count in range(n):
        x_new = np.zeros_like(x) 
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:]) 
            x_new[i] = (B[i] - s1 - s2) / A[i, i]
        if np.allclose(x, x_new, tol): 
            break
        x = x_new 
    return x

# We will use 25 iterations in our jacobi function to find the values of x.

A = np.array([[10., -1., 2., 0.], 
              [-1., 11., -1., 3.], 
              [2., -1., 10., -1.], 
              [0.0, 3., -1., 8.]]) 
B = np.array([6., 25., -11., 15.])
n = 25

x = jacobi(A, B, n)
print("x =", x)

x = [ 1.  2. -1.  1.]


### The Gauss-Seidel method

$$
A=\begin{bmatrix} a& b& c& d \\ e& f& g& h \\ i& j& k& l \\ m& n& o& p \end{bmatrix}=
\begin{bmatrix} a& 0& 0& 0 \\ e& f& 0& 0 \\ i& j& k& 0 \\ m& n& o& p \end{bmatrix}*
\begin{bmatrix} 0& b& c& d \\ 0& 0& g& h \\ 0& 0& 0& l \\ 0& 0& 0& 0 \end{bmatrix}
$$

The solution is then obtained iteratively as follows:
$$Ax = B$$

$$(L +U )x = B$$

$$Lx = B − Ux$$

$$x_{n+1} =L−1(B−Ux_n)$$

**The elements of xn can be overwritten in each iteration in order to compute xn+1.**

In [77]:
""" Solve Ax=B with the Gauss-Seidel method """ 
import numpy as np
def gauss(A, B, n, tol=1e-10):
    L = np.tril(A) # Returns the lower triangular matrix of A 
    U = A - L # Decompose A = L + U
    L_inv = np.linalg.inv(L)
    x = np.zeros_like(B)
    for i in range(n):
        Ux = np.dot(U, x)
        x_new = np.dot(L_inv, B - Ux)
        if np.allclose(x, x_new, tol): 
            break
        x = x_new 
    return x

A = np.array([[10., -1., 2., 0.], 
              [-1., 11., -1., 3.], 
              [2., -1., 10., -1.],
              [0.0, 3., -1., 8.]]) 
B = np.array([6., 25., -11., 15.])
n = 100
x = gauss(A, B, n)

print ("x =", x)

x = [ 1.  2. -1.  1.]
